**EXERCISE-3**

In [1]:
!pip install -q pyomo

     |████████████████████████████████| 9.1 MB 5.1 MB/s 
     |████████████████████████████████| 49 kB 5.4 MB/s 


In [2]:
from pyomo.environ import *

In [3]:
import numpy as np

In [4]:
import pandas as pd

#Que.1

The general optimization problem is:
$$\text{min} \ \  \sum_{i=1}^{6} |P_i-\tilde{P_i}| \\
\text{s.t.  } 
\tilde{P_i}=b_0+b_1L_i+b_2E_i \ \ \forall i \in \{1,2,...,6\}  \\
b_0 \geq 0$$ \\

No, above problem is not linear because objective function consist a modulus function which is a piecewise function.

#Que.2

**Approach-1**

let $|P_i-\tilde{P_i}|=z_i$

then using the result that $|x|$ can be written as $\text{min } z \ \  \ \text{s.t.}\ \ z\geq x$ and $z\geq-x$
using this property the optimization problem can be written as:
$$\text{min } \sum_{i=1}^6 z_i\\\text{s.t. }
 \tilde{P_i}=b_0+b_1L_i+b_2E_i \ \ \forall i \in \{1,2,...,6\}  \\
 z_i\geq P_i-\tilde{P_i}\ \ \forall i \in \{1,2,...,6\}  \\
 z_i\geq \tilde{P_i}-P_i\ \ \forall i \in \{1,2,...,6\}  \\
 b_0 \geq 0\\
 z_i\geq 0 \ \ \forall i \in \{1,2,...,6\}
$$

after putting the value of $\tilde{P_i}$ this problem can be written as:
$$\text{min } \sum_{i=1}^6 z_i\\\text{s.t. }
 z_i+b_0+b_1L_i+b_2E_i \geq P_i\ \ \forall i \in \{1,2,...,6\}  \\
 z_i-b_0-b_1L_i-b_2E_i \geq -P_i\ \ \forall i \in \{1,2,...,6\}  \\
 b_0 \geq 0\\
 z_i\geq 0 \ \ \forall i \in \{1,2,...,6\}
$$ 

**Approach-2**

let $P_i-\tilde{P_i}=x_i-y_i$ where $x_i\geq0 $ and $y_i\geq0 \ \ \forall i \in \{1,2,...,6\} $

then $|P_i-\tilde{P_i}|=|x_i-y_i|=x_i+y_i $ where $x_i\geq0 $ and $y_i\geq0 \ \ \forall i \in \{1,2,...,6\}$

Now the constraint $\tilde{P_i}=b_0+b_1L_i+b_2E_i $ can be written as :

$$P_i-\tilde{P_i}=P_i-b_0-b_1L_i-b_2E_i \\
\implies x_i-y_i+b_0+b_1L_i+b_2E_i=P_i $$

now our optimization problem can be written as:

$$\text{Min  } \sum_{i=1}^{6} (x_i+y_i) \\
\text{s.t.} \\
x_i-y_i+b_0+b_1L_i+b_2E_i=P_i \ \ \forall i \\
x_i \geq 0, y_i \geq 0 \ \ \forall i \in \{1,2,...,6\}\\\
b_0\geq 0
$$


**Solving using Approach-1**

In [5]:
data_csv=pd.read_csv('lab6_ex3.csv')

In [6]:
data_csv.columns

Index(['warehouse', 'seling price', 'lot size', 'elevation'], dtype='object')

In [7]:
data_csv['warehouse']

0    1
1    2
2    3
3    4
4    5
5    6
Name: warehouse, dtype: int64

In [8]:
data_csv.index

RangeIndex(start=0, stop=6, step=1)

In [9]:
len(data_csv.index)

6

In [10]:
model=ConcreteModel()

In [11]:
N=len(data_csv.index)#no. of variables

In [12]:
col_indices=np.arange(N)
b_indices=np.arange(3)

In [13]:
model.z=Var(col_indices,domain=NonNegativeReals)
model.b=Var(b_indices)

In [14]:
model.objective=Objective(expr=summation(model.z),sense=minimize)

In [15]:
model.constraints=ConstraintList()

In [16]:
for i in col_indices:
  model.constraints.add(model.z[i]+model.b[0]+model.b[1]*data_csv['lot size'][i]+model.b[2]*data_csv['elevation'][i] >= data_csv['seling price'][i])

In [17]:
for i in col_indices:
  model.constraints.add(model.z[i]-model.b[0]-model.b[1]*data_csv['lot size'][i]-model.b[2]*data_csv['elevation'][i] >= -data_csv['seling price'][i])

In [18]:
model.b[0].setlb(0)

In [19]:
model.pprint()

3 Set Declarations
    b_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
    z_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

2 Var Declarations
    b : Size=3, Index=b_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :  None :  None :  None : False :  True :  Reals
          2 :  None :  None :  None : False :  True :  Reals
    z : Size=6, Index=z_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : Fa

In [20]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 148492 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

In [21]:
opt_cbc=SolverFactory('cbc')

In [22]:
result1=opt_cbc.solve(model)


**optimal value,values of new and original decisoion variables,solver status and solver termination condition:-**

In [23]:
print('Solver status:', result1.solver.status)
print('Solver termination condition:',result1.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [24]:
print('\nOptimal sum of residuals = ', model.objective())

print('\nNew Decision Variables')
for i in col_indices:
  print('z[',i+1,']=', model.z[i].value)

print('\nOriginal Decision Variables')
for i in b_indices:
  print('b[',i,']=', model.b[i].value)
for i in col_indices:
  print(u'P\u0304[',i,']=', model.b[0].value+model.b[1].value*data_csv['lot size'][i] +model.b[2].value*data_csv['elevation'][i] )

print('\nConstraints')
model.constraints.display()


Optimal sum of residuals =  125454.545

New Decision Variables
z[ 1 ]= 38818.182
z[ 2 ]= 0.0
z[ 3 ]= 54272.727
z[ 4 ]= 32363.636
z[ 5 ]= 0.0
z[ 6 ]= 0.0

Original Decision Variables
b[ 0 ]= 104181.82
b[ 1 ]= 67.272727
b[ 2 ]= -356.36364
P̄[ 0 ]= 136181.81860000006
P̄[ 1 ]= 149999.9996
P̄[ 2 ]= 179272.72650000005
P̄[ 3 ]= 122363.63760000002
P̄[ 4 ]= 80000.00090000003
P̄[ 5 ]= 120000.00150000001

Constraints
constraints : Size=12
    Key : Lower     : Body                : Upper
      1 :  175000.0 :  175000.00060000003 :  None
      2 :  150000.0 :         149999.9996 :  None
      3 :  125000.0 :  233545.45350000006 :  None
      4 :   90000.0 :  154727.27360000001 :  None
      5 :   80000.0 :   80000.00090000003 :  None
      6 :  120000.0 :  120000.00150000001 :  None
      7 : -175000.0 :  -97363.63660000001 :  None
      8 : -150000.0 :        -149999.9996 :  None
      9 : -125000.0 : -124999.99950000003 :  None
     10 :  -90000.0 :  -90000.00160000002 :  None
     11 :  -80000

**Solving using Approach-2**

In [25]:
model2=ConcreteModel()
model2.x=Var(col_indices,domain=NonNegativeReals)
model2.y=Var(col_indices,domain=NonNegativeReals)
model2.b=Var(b_indices)


In [26]:
model2.objective=Objective(expr=summation(model2.x)+summation(model2.y),sense=minimize)

In [27]:
model2.constraints=ConstraintList()

In [28]:
for i in col_indices:
  model2.constraints.add(model2.x[i]-model2.y[i]+model2.b[0]+model2.b[1]*data_csv['lot size'][i]+model2.b[2]*data_csv['elevation'][i] == data_csv['seling price'][i])

In [29]:
model2.pprint()

4 Set Declarations
    b_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {1, 2, 3, 4, 5, 6}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}
    y_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

3 Var Declarations
    b : Size=3, Index=b_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :  None :  None :  None : False :  True :  Reals
          1 :  None :  None :  None : False :  True :  Reals
          2 :  None :  None :  None : False :  True :  Reals
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fi

**optimal value,values of new and original decisoion variables,solver status and solver termination condition:-**

In [30]:
result2=opt_cbc.solve(model2)
print(result2)
print(f"Solver Status: {result2.solver.status}")
print(f"The termination condition is :{result2.solver.termination_condition}")


Problem: 
- Name: unknown
  Lower bound: 125454.5455
  Upper bound: 125454.5455
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 16
  Number of nonzeros: 12
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 8
  Error rc: 0
  Time: 0.03184771537780762
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver Status: ok
The termination condition is :optimal


In [31]:
print('\nOptimal sum of residuals = ', model2.objective())

print('\nNew Decision Variables')
for i in col_indices:
  print('x[',i+1,']=', model2.x[i].value)
for i in col_indices:
  print('y[',i+1,']=', model2.y[i].value)

print('\nOriginal Decision Variables')
for i in b_indices:
  print('b[',i,']=', model2.b[i].value)
for i in col_indices:
  print(u'P\u0304[',i,']=', model2.b[0].value+model2.b[1].value*data_csv['lot size'][i] +model2.b[2].value*data_csv['elevation'][i] )

print('\nConstraints')
model2.constraints.display()


Optimal sum of residuals =  125454.545

New Decision Variables
x[ 1 ]= 38818.182
x[ 2 ]= 0.0
x[ 3 ]= 0.0
x[ 4 ]= 0.0
x[ 5 ]= 0.0
x[ 6 ]= 0.0
y[ 1 ]= 0.0
y[ 2 ]= 0.0
y[ 3 ]= 54272.727
y[ 4 ]= 32363.636
y[ 5 ]= 0.0
y[ 6 ]= 0.0

Original Decision Variables
b[ 0 ]= 104181.82
b[ 1 ]= 67.272727
b[ 2 ]= -356.36364
P̄[ 0 ]= 136181.81860000006
P̄[ 1 ]= 149999.9996
P̄[ 2 ]= 179272.72650000005
P̄[ 3 ]= 122363.63760000002
P̄[ 4 ]= 80000.00090000003
P̄[ 5 ]= 120000.00150000001

Constraints
constraints : Size=6
    Key : Lower    : Body               : Upper
      1 : 175000.0 : 175000.00060000003 : 175000.0
      2 : 150000.0 :        149999.9996 : 150000.0
      3 : 125000.0 : 124999.99950000003 : 125000.0
      4 :  90000.0 :  90000.00160000002 :  90000.0
      5 :  80000.0 :  80000.00090000003 :  80000.0
      6 : 120000.0 : 120000.00150000001 : 120000.0


I observed that both approaches gave exactly same solution.